In [2]:
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, GRU
from keras.callbacks import Callback
from keras import metrics
from sklearn.impute import KNNImputer

In [4]:
with open('/content/dailymonthly.csv', encoding='utf-8-sig') as file_obj:
    # Create reader object by passing the file object to DictReader method
    reader_obj = csv.DictReader(file_obj)

    # Open a file to save the results
    with open("/content/GRU_IMD_RDI.csv", "w+") as f:
        # Define a csv writer object
        writer = csv.writer(f)
        writer.writerow(['Loc', 'mse', 'rmse', 'mae'])

        # Iterate over each row in the csv file using reader object
        for row in reader_obj:
            data = row['daily']
            print(row)

            imd = pd.read_csv('/content/IMD/' + data + '.csv').drop(['Unnamed: 0', 'Year'], axis=1)
            imd = imd.replace([np.inf, -np.inf], np.nan)

            X = imd.iloc[5:, 0:4]
            Y = imd.iloc[5:, 3:4]

            imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
            imputer.fit(Y)
            Y = imputer.transform(Y)
            X['RDIn'] = Y
            Y = X.iloc[:, 3]
            Ya = Y.iloc[10:]

            X_train = np.zeros((30, 10))
            X_test = np.zeros((20, 10))
            for i in range(30):
                X_train[i, :] = Y[i:i + 10]
            for i in range(20):
                X_test[i, :] = Y[i + 30:i + 30 + 10]

            X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
            Y_train = np.array(Ya.iloc[0:30])
            Y_train = Y_train.reshape(Y_train.shape[0], 1, 1)

            X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
            Y_test = np.array(Ya.iloc[30:])
            Y_test = Y_test.reshape(Y_test.shape[0], 1, 1)

            model = Sequential()
            model.add(GRU(25, return_sequences=True,dropout=0.1))
            model.add(GRU(35, return_sequences=True,dropout=0.1))
            model.add(GRU(45, return_sequences=True,dropout=0.1))
            model.add(Dense(8))
            model.add(Dense(1))
            #model detals can be changed as per requirement.

            model.compile(
                loss='mae',
                optimizer=tf.keras.optimizers.legacy.Adam(
                    learning_rate=0.003,
                    decay=0.0
                    ),
                metrics=[tf.keras.metrics.RootMeanSquaredError(), 'mse', 'mae']
                )

            history = model.fit(
                X_train,
                Y_train,
                batch_size=32,
                epochs=10,
                verbose=1,
                callbacks=[]
                )
            Y_pred = model.predict(X_test)

            mse = history.history['mse'][-1]  # Mean Squared Error
            mse = round(mse , 5)
            rmse = np.sqrt(mse)  # Root Mean Squared Error
            rmse = round(rmse , 5)
            mae = history.history['mae'][-1]  # Mean Absolute Error
            mae = round(mae , 5)

            # Write the results to the file
            writer.writerow([data, mse, rmse, mae])

{'daily': 'Ajmer', 'monthly': 'Ajmer'}
Epoch 1/10
1/1 [==============================] - 14s 14s/step - loss: 1.0117 - root_mean_squared_error: 1.1255 - mse: 1.2668 - mae: 1.0117
Epoch 2/10
1/1 [==============================] - 0s 44ms/step - loss: 0.7561 - root_mean_squared_error: 0.9023 - mse: 0.8142 - mae: 0.7561
Epoch 3/10
1/1 [==============================] - 0s 35ms/step - loss: 0.5597 - root_mean_squared_error: 0.7330 - mse: 0.5372 - mae: 0.5597
Epoch 4/10
1/1 [==============================] - 0s 35ms/step - loss: 0.5404 - root_mean_squared_error: 0.6585 - mse: 0.4336 - mae: 0.5404
Epoch 5/10
1/1 [==============================] - 0s 34ms/step - loss: 0.5834 - root_mean_squared_error: 0.6980 - mse: 0.4872 - mae: 0.5834
Epoch 6/10
1/1 [==============================] - 0s 65ms/step - loss: 0.5779 - root_mean_squared_error: 0.6934 - mse: 0.4807 - mae: 0.5779
Epoch 7/10
1/1 [==============================] - 0s 34ms/step - loss: 0.5442 - root_mean_squared_error: 0.6592 - mse: 0.

1/1 [==============================] - 1s 1s/step
{'daily': 'Banswara', 'monthly': 'Banswara'}
Epoch 1/10
1/1 [==============================] - 7s 7s/step - loss: 1.0850 - root_mean_squared_error: 1.1330 - mse: 1.2837 - mae: 1.0850
Epoch 2/10
1/1 [==============================] - 0s 24ms/step - loss: 0.7819 - root_mean_squared_error: 0.8509 - mse: 0.7241 - mae: 0.7819
Epoch 3/10
1/1 [==============================] - 0s 28ms/step - loss: 0.5118 - root_mean_squared_error: 0.6139 - mse: 0.3769 - mae: 0.5118
Epoch 4/10
1/1 [==============================] - 0s 25ms/step - loss: 0.3962 - root_mean_squared_error: 0.5070 - mse: 0.2571 - mae: 0.3962
Epoch 5/10
1/1 [==============================] - 0s 25ms/step - loss: 0.4521 - root_mean_squared_error: 0.5532 - mse: 0.3060 - mae: 0.4521
Epoch 6/10
1/1 [==============================] - 0s 25ms/step - loss: 0.4916 - root_mean_squared_error: 0.5841 - mse: 0.3412 - mae: 0.4916
Epoch 7/10
1/1 [==============================] - 0s 27ms/step - lo

1/1 [==============================] - 2s 2s/step
{'daily': 'Baran', 'monthly': 'Baran'}
Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 0.8817 - root_mean_squared_error: 0.9106 - mse: 0.8293 - mae: 0.8817
Epoch 2/10
1/1 [==============================] - 0s 27ms/step - loss: 0.5975 - root_mean_squared_error: 0.6514 - mse: 0.4244 - mae: 0.5975
Epoch 3/10
1/1 [==============================] - 0s 26ms/step - loss: 0.3735 - root_mean_squared_error: 0.4676 - mse: 0.2187 - mae: 0.3735
Epoch 4/10
1/1 [==============================] - 0s 27ms/step - loss: 0.4426 - root_mean_squared_error: 0.5273 - mse: 0.2781 - mae: 0.4426
Epoch 5/10
1/1 [==============================] - 0s 31ms/step - loss: 0.4593 - root_mean_squared_error: 0.5293 - mse: 0.2802 - mae: 0.4593
Epoch 6/10
1/1 [==============================] - 0s 26ms/step - loss: 0.4020 - root_mean_squared_error: 0.4757 - mse: 0.2263 - mae: 0.4020
Epoch 7/10
1/1 [==============================] - 0s 39ms/step - loss: 0.